## Nama : Alisha Zahra Saadiya
## NIM : 2501971742
## Link Video : https://binusianorg-my.sharepoint.com/personal/alisha_saadiya_binus_ac_id/_layouts/15/guestaccess.aspx?share=ESzraNsI4D9MrN4DII5e5dsBxQum693OeeZwwwiRKEL5sA&nav=eyJyZWZlcnJhbEluZm8iOnsicmVmZXJyYWxBcHAiOiJPbmVEcml2ZUZvckJ1c2luZXNzIiwicmVmZXJyYWxBcHBQbGF0Zm9ybSI6IldlYiIsInJlZmVycmFsTW9kZSI6InZpZXciLCJyZWZlcnJhbFZpZXciOiJNeUZpbGVzTGlua0NvcHkifX0&e=PTnwym

In [ ]:
pip install rouge-score

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import networkx as nx
from rouge_score import rouge_scorer

In [ ]:
# Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Alisha
[nltk_data]     Zahra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def preprocess_text(content):
    sentences = content.split(". ")
    processed_sentences = [sentence.replace("[^a-zA-Z]", " ").split(" ") for sentence in sentences]
    return processed_sentences


In [ ]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

In [ ]:
def build_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [ ]:
def generate_summary(content, top_n):
    stop_words = stopwords.words('indonesian')
    summarize_text = []

    sentences = preprocess_text(content)
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    return ". ".join(summarize_text)

In [ ]:
def evaluate_summary(reference_summary, generated_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores

In [ ]:
# Read data
df = pd.read_csv("dataset.csv")

In [ ]:
summaries = {'top_3': [], 'top_5': []}

In [ ]:
# Generate summaries
for i, row in df.iterrows():
    content = row['content']
    summary_top_3 = generate_summary(content, 3)
    summary_top_5 = generate_summary(content, 5)
    summaries['top_3'].append(summary_top_3)
    summaries['top_5'].append(summary_top_5)

    print(f"\nOriginal Content:\n{content}\n")
    print(f"Top 3 Summary:\n{summary_top_3}\n")
    print(f"Top 5 Summary:\n{summary_top_5}\n")


Original Content:
TEMPO.CO, Jakarta -Ketua Umum Partai Gerindra, Prabowo Subianto, menerima kunjungan Ketua Umum Partai NasDem, Surya Paloh, di kediamannya di Hambalang, Bogor, pada Ahad, 5 Maret 2023. Dalam pertemuan itu, Prabowo menyatakan bahwa mereka sepakat untuk saling menghormati keputusan politik masing-masing.Prabowo mengakui adanya pembicaraan soal politik dalam pertemuan itu. Dia menyatakan bahwa dirinya maupun Surya Paloh sepakat untuk bersaing secara sehat dalam Pemilu dan Pilpres 2024.Menteri Pertahanan itu menyatakan bahwa mereka sepakat untuk mengedepankan perdamaian dan kerukunan bangsa dalam persaingan memenangkan pesta demokrasi tersebut."Intinya tadi pembicaraan sangat cair setelah kita makan siang kemudian kita tadi minum kopi dan saya bicara cukup luas. Kami dapat suatu kesimpulan untuk sepakat saling menghormati keputusan politik masing-masing," kata Prabowo usai pertemuan itu."Kami sepakat karena kami ingin suasana bangsa dan negara selalu dalam keadaan damai, 

In [ ]:
# Evaluate summaries using ROUGE score
rouge_scores = {'top_3': [], 'top_5': []}
for i, row in df.iterrows():
    reference_summary = row['summary']
    rouge_scores['top_3'].append(evaluate_summary(reference_summary, summaries['top_3'][i]))
    rouge_scores['top_5'].append(evaluate_summary(reference_summary, summaries['top_5'][i]))

In [ ]:
evaluate_summary(reference_summary, summaries['top_3'][i])

{'rouge1': Score(precision=0.2777777777777778, recall=0.2631578947368421, fmeasure=0.27027027027027023),
 'rouge2': Score(precision=0.17142857142857143, recall=0.16216216216216217, fmeasure=0.16666666666666669),
 'rougeL': Score(precision=0.25, recall=0.23684210526315788, fmeasure=0.24324324324324326)}

- ROUGE-1:

    Precision: 0.2777
    
    Ini berarti dari semua unigram (kata tunggal) yang ada dalam summary yang dihasilkan, sekitar 27.77% di antaranya juga ada dalam summary pada dataset.

    Recall: 0.2632
    
    Ini berarti dari semua unigram yang ada dalam summary pada dataset, sekitar 26.32% di antaranya juga ada dalam summary yang dihasilkan.
    
    F1-score: 0.2703
    
    Ini adalah harmonic mean dari precision dan recall untuk ROUGE-1, yang memberikan nilai gabungan dari kedua metrik tersebut.

- ROUGE-2:

    Precision: 0.1714

    Ini berarti dari semua bigram (pasangan kata) yang ada dalam summary yang dihasilkan, sekitar 17.14% di antaranya juga ada dalam summary pada dataset.
    
    Recall: 0.1622
    
    Ini berarti dari semua bigram yang ada dalam summary pada dataset, sekitar 16.22% di antaranya juga ada dalam summary yang dihasilkan.

    F1-score: 0.1667

    Ini adalah harmonic mean dari precision dan recall untuk ROUGE-2.

- ROUGE-L:

    Precision: 0.2500
    
    Ini berarti dari semua unit LCS (Longest Common Subsequence) antara summary yang dihasilkan dan summary pada dataset, sekitar 25.00% di antaranya adalah unit LCS dari summary pada dataset.
    
    Recall: 0.2368
    
    Ini berarti dari semua unit LCS antara summary yang dihasilkan dan summary pada dataset, sekitar 23.68% di antaranya adalah unit LCS dari summary yang dihasilkan.
    
    F1-score: 0.2432
    
    Ini adalah harmonic mean dari precision dan recall untuk ROUGE-L.

In [ ]:
evaluate_summary(reference_summary, summaries['top_5'][i])

{'rouge1': Score(precision=0.3050847457627119, recall=0.47368421052631576, fmeasure=0.3711340206185567),
 'rouge2': Score(precision=0.20689655172413793, recall=0.32432432432432434, fmeasure=0.25263157894736843),
 'rougeL': Score(precision=0.22033898305084745, recall=0.34210526315789475, fmeasure=0.26804123711340205)}

- ROUGE-1:

    Precision: 0.3051
    
    Sekitar 30.51% dari semua unigram (kata tunggal) yang ada dalam summary yang dihasilkan juga ada dalam summary pada dataset.
    
    Recall: 0.4737
    
    Sekitar 47.37% dari semua unigram yang ada dalam summary pada dataset juga ada dalam summary yang dihasilkan.
    
    F1-score: 0.3711
    
    Harmonic mean dari precision dan recall untuk ROUGE-1, memberikan nilai gabungan dari kedua metrik tersebut.

- ROUGE-2:

    Precision: 0.2069
    
    Sekitar 20.69% dari semua bigram (pasangan kata) yang ada dalam summary yang dihasilkan juga ada dalam summary pada dataset.
    
    Recall: 0.3243
    
    Sekitar 32.43% dari semua bigram yang ada dalam summary pada dataset juga ada dalam summary yang dihasilkan.
    
    F1-score: 0.2526
    
    Harmonic mean dari precision dan recall untuk ROUGE-2.

- ROUGE-L:

    Precision: 0.2203

    Sekitar 22.03% dari semua unit LCS (Longest Common Subsequence) antara summary yang dihasilkan dan summary pada dataset adalah unit LCS dari summary pada dataset.
    
    Recall: 0.3421
    
    Sekitar 34.21% dari semua unit LCS antara summary yang dihasilkan dan summary pada dataset adalah unit LCS dari summary yang dihasilkan.
    
    F1-score: 0.2680
    
    Harmonic mean dari precision dan recall untuk ROUGE-L.

In [ ]:
# Calculate average ROUGE scores
avg_rouge_scores_top_3 = {key: np.mean([score[key].fmeasure for score in rouge_scores['top_3']]) for key in rouge_scores['top_3'][0]}
avg_rouge_scores_top_5 = {key: np.mean([score[key].fmeasure for score in rouge_scores['top_5']]) for key in rouge_scores['top_5'][0]}

print("Average ROUGE scores for Top 3 summary sentences:")
print(avg_rouge_scores_top_3)

print("Average ROUGE scores for Top 5 summary sentences:")
print(avg_rouge_scores_top_5)

Average ROUGE scores for Top 3 summary sentences:
{'rouge1': 0.29429625768298534, 'rouge2': 0.1478015448603684, 'rougeL': 0.21173379983448637}
Average ROUGE scores for Top 5 summary sentences:
{'rouge1': 0.2922257132296727, 'rouge2': 0.16196305976452546, 'rougeL': 0.18969174733897684}


- ROUGE-1
    
    Top 3 Summary Sentences: 0.294
    
    Top 5 Summary Sentences: 0.292
    
    ROUGE-1 mengukur persentase unigram (kata tunggal) yang cocok antara summary yang dihasilkan dan kolom summary pada dataset. Nilai ROUGE-1 untuk top 3 dan top 5 summary sentences hampir sama, menunjukkan bahwa pemilihan tiga atau lima kalimat terbaik menghasilkan proporsi kata yang hampir sama dalam kecocokannya dengan summary pada dataset.
- ROUGE-2
    
    Top 3 Summary Sentences: 0.148
    
    Top 5 Summary Sentences: 0.162
    
    ROUGE-2 mengukur persentase bigram (dua kata berurutan) yang cocok. Nilai ROUGE-2 sedikit lebih tinggi untuk top 5 summary yang dihasilkan dibandingkan dengan top 3, menunjukkan bahwa memasukkan lebih banyak kalimat membantu menangkap lebih banyak konteks dua kata berurutan dari summary pada dataset.
- ROUGE-L
    
    Top 3 Summary Sentences: 0.212
    
    Top 5 Summary Sentences: 0.190
    
    ROUGE-L mengukur kesamaan berbasis subsekuensi terpanjang yang ada di antara summary yang dihasilkan dan summary pada dataset. Nilai ROUGE-L untuk top 3 summary yang dihasilkan lebih tinggi dibandingkan dengan top 5. Ini menunjukkan bahwa tiga kalimat terbaik memiliki struktur yang lebih mirip dengan summary pada dataset dibandingkan dengan lima kalimat terbaik.  

Berdasarkan hasil ini, pemilihan antara top 3 dan top 5 summary yang dihasilkan dapat disesuaikan dengan kebutuhan spesifik. Jika fokusnya adalah mendapatkan ringkasan yang lebih padat dan struktural, top 3 lebih cocok. Namun, jika summary yang lebih informatif dan kontekstual diinginkan, top 5 lebih tepat.